# Linear Solver Performance: XDG Stokes, Single Core
### Part 1, Benchmark Setup and Execution

In [1]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

The below script needs to be able to find the current output cell; this is an easy method to get it.

Execution Date/time is 5/16/2022 11:59:43 AM


In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [3]:
wmg.Init("LinslvPerfSer");
wmg.AllJobs

Project name is set to 'LinslvPerfSer'.
Opening existing database '\\fdygitrunner\ValidationTests\LinslvPerfSer'.


In [4]:
var myBatch = GetDefaultQueue();
myBatch

RuntimeLocation,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
win\amd64,\\fdygitrunner\ValidationTests,True,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2, hpcluster3, fdygitrunner ]",Normal,True,[ \\fdygitrunner\ValidationTests == ]


In [5]:
// for examination on the local workstation, 
//OpenDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes");

In [6]:
/*
// extract the control object to reproduce specific simulation on local workstation
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\BenchControls";
foreach(var s in wmg.Sessions) {
    string name = s.Name;
    Console.Write(name + ": ");
    try {
        var ctrl = s.GetControl();
        var txt = ctrl.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

## Utility definitions

In [7]:
static class Utils {
    // DOF per cell in 3D for one variable
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

Note: The dimension of the domain $(-1,1)^3$ are assumed to be **centimeters**!
In this benchmark, realistic physical values are used, i.e. densities and viscosities of 
- water for Phase A (inside the droplet)
- air for Phase B (surounding)
The droplet is assumed to be in the millimeter range; therefore, all lenght-, area-, and volume-related properties 
must also be set in centimeters, e.g. the density of water is $10^{-3} \text{kg}/\text{cm}^3$.

In [8]:
int[] Resolutions_3D = new int[] { 8, 16, 24, 32, 48, 64 };
//int[] Resolutions_3D = new int[] { 2 };
IGridInfo[] grids = new IGridInfo[Resolutions_3D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] zNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "-XdgStokes_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(GridName)); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            if (Math.Abs(X[0] - (-1)) <= 1.0e-8)
                return "wall_left";
            if (Math.Abs(X[0] - (+1)) <= 1.0e-8)
                return "wall_right";
            if (Math.Abs(X[1] - (-1)) <= 1.0e-8)
                return "wall_front";
            if (Math.Abs(X[1] - (+1)) <= 1.0e-8)
                return "wall_back";
            if (Math.Abs(X[2] - (-1)) <= 1.0e-8)
                return "wall_top";
            if (Math.Abs(X[2] - (+1)) <= 1.0e-8)
                return "wall_bottom";
            throw new ArgumentException("unknown wall");
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

Opening existing database 'C:\BoSSStests\LinslvPerfSer'.
Found Grid: { Guid = 27d0a4c6-8abf-4044-9aa5-3ee5617796dc; Name = LinslvPerfSer-XdgStokes_J512; Cell Count = 512; Dim = 3 }
Found Grid: { Guid = 1ef5cde5-d32c-491e-9781-038a2a1e6c76; Name = LinslvPerfSer-XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
Found Grid: { Guid = 97c06871-8533-4d3a-b5b4-944abebb1107; Name = LinslvPerfSer-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
Found Grid: { Guid = 99a1067c-b759-4608-8078-3249ce5a0a9b; Name = LinslvPerfSer-XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
Found Grid: { Guid = 135c2dd5-2032-4a40-83d9-483d39249b02; Name = LinslvPerfSer-XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
Found Grid: { Guid = 7b44677e-e081-42e6-9c71-28e0a6aeeca8; Name = LinslvPerfSer-XdgStokes_J262144; Cell Count = 262144; Dim = 3 }


In [9]:
grids

#0: { Guid = 27d0a4c6-8abf-4044-9aa5-3ee5617796dc; Name = LinslvPerfSer-XdgStokes_J512; Cell Count = 512; Dim = 3 }
#1: { Guid = 1ef5cde5-d32c-491e-9781-038a2a1e6c76; Name = LinslvPerfSer-XdgStokes_J4096; Cell Count = 4096; Dim = 3 }
#2: { Guid = 97c06871-8533-4d3a-b5b4-944abebb1107; Name = LinslvPerfSer-XdgStokes_J13824; Cell Count = 13824; Dim = 3 }
#3: { Guid = 99a1067c-b759-4608-8078-3249ce5a0a9b; Name = LinslvPerfSer-XdgStokes_J32768; Cell Count = 32768; Dim = 3 }
#4: { Guid = 135c2dd5-2032-4a40-83d9-483d39249b02; Name = LinslvPerfSer-XdgStokes_J110592; Cell Count = 110592; Dim = 3 }
#5: { Guid = 7b44677e-e081-42e6-9c71-28e0a6aeeca8; Name = LinslvPerfSer-XdgStokes_J262144; Cell Count = 262144; Dim = 3 }


## Setup Control Object for a Solver Run

In [10]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [11]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Setup of Parameter Study

Polynomial degrees to test:

In [12]:
int[] PolyDegS = new int[] {2, 3 ,5};

Solvers which we want to instrument:

In [13]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.direct_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    //LinearSolverCode.pMultigrid
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [14]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.direct_pardiso:
        case LinearSolverCode.direct_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 3000000; // Up to 3 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [15]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [16]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*3 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("XdgStokes-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    // Species A: Water; Species B: Air
    C.PhysicalParameters.rho_A             = 1e-3; //     kg / cm³
    C.PhysicalParameters.rho_B             = 1.2e-6; //   kg / cm³
    C.PhysicalParameters.mu_A              = 1e-5; //      kg / cm / sec
    C.PhysicalParameters.mu_B              = 17.1e-8; //   kg / cm / sec
    C.PhysicalParameters.Sigma             = 72.75e-3; // kg / sec²   
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
    // Dont know
    // ============
    
    double r     = 0.5;
    double nonsp = 0.5;

    C.AddInitialValue("Phi", new Formula($"X => (X[0]/{r*nonsp}).Pow2() + (X[1]/{r}).Pow2() + (X[2]/{r}).Pow2() - 1", false));
    
    C.LSContiProjectionMethod = BoSSS.Solution.LevelSetTools.ContinuityProjectionOption.None;
    //C.CutCellQuadratureType   = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.ComputeEnergyProperties = false;

    
    // Solver Stuff
    // ============
    
    C.LinearSolver           = solver.GetConfig();
    if(C.LinearSolver is IterativeSolverConfig isc) {
        isc.ConvergenceCriterion = 1e-8;
    }
    C.NoOfMultigridLevels = 100;
    C.LevelSet_ConvergenceCriterion     = 1e-6;

    //C.Option_LevelSetEvolution                          = LevelSetEvolution.FastMarching;
    C.AdvancedDiscretizationOptions.SST_isotropicMode   = SurfaceStressTensor_IsotropicMode.Curvature_Projected;
    //C.AdvancedDiscretizationOptions.ViscosityMode       = ViscosityMode.Standard;
    C.AdvancedDiscretizationOptions.FilterConfiguration = CurvatureAlgorithms.FilterConfiguration.NoFilter;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

setting up: XdgStokes-J512_p2_direct_pardiso
setting up: XdgStokes-J4096_p2_direct_pardiso
setting up: XdgStokes-J13824_p2_direct_pardiso
setting up: XdgStokes-J512_p3_direct_pardiso
setting up: XdgStokes-J4096_p3_direct_pardiso
setting up: XdgStokes-J13824_p3_direct_pardiso
setting up: XdgStokes-J512_p5_direct_pardiso
setting up: XdgStokes-J4096_p5_direct_pardiso
setting up: XdgStokes-J512_p2_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p2_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p2_exp_gmres_levelpmg
setting up: XdgStokes-J32768_p2_exp_gmres_levelpmg
setting up: XdgStokes-J512_p3_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p3_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p3_exp_gmres_levelpmg
setting up: XdgStokes-J32768_p3_exp_gmres_levelpmg
setting up: XdgStokes-J512_p5_exp_gmres_levelpmg
setting up: XdgStokes-J4096_p5_exp_gmres_levelpmg
setting up: XdgStokes-J13824_p5_exp_gmres_levelpmg
setting up: XdgStokes-J512_p2_exp_Kcycle_schwarz
setting up: XdgStokes-J4096_p2_e

Total number of simulations:

In [17]:
controls.Count

30

In [18]:
// Assert that the location of the fluid phases is as desired:
NUnit.Framework.Assert.Negative(controls[0].InitialValues["Phi"].Evaluate(new double[]{0,0.0,0}, 0.0), "phase A (negative) must be inside");
NUnit.Framework.Assert.Positive(controls[0].InitialValues["Phi"].Evaluate(new double[]{0,4.0,0}, 0.0), "phase B (positive) must be inside");

In [19]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    if(ctrl.LinearSolver is OrthoMGSchwarzConfig osc)
        NUnit.Framework.Assert.Greater(osc.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

In [20]:
//string path = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\Xstokes";
//foreach(var ctrl in controls) {
//    ctrl.SaveToFile(System.IO.Path.Combine(path, ctrl.SessionName + ".obj"));
//}

## Launch Jobs

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    ctrl.RunBatch();
}

 Submitting: XdgStokes-J512_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	XdgStokes-J512_p2_direct_pardiso	04/15/2022 09:14:28	bb4011d8..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: XdgStokes-J4096_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	XdgStokes-J4096_p2_direct_pardiso	04/15/2022 09:15:02	b103e7b3..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: XdgStokes-J13824_p2_direct_pardiso
Info: Found successful session "LinslvPerfSer	XdgStokes-J13824_p2_direct_pardiso	04/15/2022 09:15:40	fb4907f7..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
 Submitting: XdgStokes-J512_p3_direct_pardiso
Info: Found successful session "LinslvPerfSer	XdgStokes-J512_p3_direct_pardiso	04/15/2022 09:16:16	b0c62aab..." -- job is marked as successful, no fur

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions

In [ ]:
//wmg.Sessions[3].Export().WithSupersampling(2).Do()

In [ ]:
var NoSuccess = controls.Select(ctrl => ctrl.GetJob()).Where(job => job.Status != JobStatus.FinishedSuccessful).ToArray();
NoSuccess

In [ ]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + fail.LatestDeployment.DeploymentDirectory.FullName);
    //Console.WriteLine(fail.LatestDeployment);

In [ ]:
/*
string PathOffset = @"C:\Users\jenkinsci\Desktop\LinSlvPerfFail-20apr22";
foreach(var fail in NoSuccess) {
    var C = fail.GetControl();
    C.savetodb = false;
    C.SaveToFile(System.IO.Path.Combine(PathOffset, fail.Name + ".obj"));
    
    string Stdout = fail.Stdout;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stdout.txt"), Stdout);
    
    string Stderr = fail.Stderr;
    System.IO.File.WriteAllText(System.IO.Path.Combine(PathOffset, fail.Name + "-stderr.txt"), Stderr);
}
*/

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.Name.Contains("XdgStokes") && Si.SuccessfulTermination == false);
FailedSessions

#### Asserting Success:

Remark: since this is currently (22 Apr. 2022) work-in-progress, we allow for some jobs to fail; 
At this intermediate milestone, I want to record (by the means of tests) what **is already working**.
Thereby, I hope I won't break the working cases while trying to fix the failing ones.

In [ ]:
var prelim_allowedFails = new[] { "XdgStokes-J13824_p5_exp_Kcycle_schwarz", 
"XdgStokes-J32768_p3_exp_Kcycle_schwarz", 
"XdgStokes-J13824_p5_exp_gmres_levelpmg", 
"XdgStokes-J32768_p3_exp_gmres_levelpmg" };

In [ ]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Where(job => !prelim_allowedFails.Contains(job.Name)).Count(), "Some Jobs Failed");

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Where(s => !prelim_allowedFails.Contains(s.Name)).Count(), "Some Sessions did not terminate successfully.");